In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import random
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices
from sklearn.model_selection import train_test_split
import lightgbm as lgbm
import pickle

In [ ]:
def load_pickle(fich):
    with open(fich,"rb") as f :
        contenu = pickle.load(f)
    return contenu

In [ ]:
path = "../input/eda-missing-values-tsne-clustering-class-imb"

In [ ]:
SEED = 1111

In [ ]:
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
print('load train start')
try:
    train = pd.read_parquet('train.parquet')
except:
    train = pd.read_csv('../input/jane-street-market-prediction/train.csv', index_col='ts_id')
    train.to_parquet('train.parquet')
print('load train end')

In [ ]:
# train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]

train.fillna(train.mean(),inplace=True)

train['action'] = ((train['resp'].values) > 0).astype(int)


#features = [c for c in train.columns if "feature" in c]
features = load_pickle(os.path.join(path,"features_names"))

#f_mean = np.mean(train[features[1:]].values,axis=0)
f_mean = load_pickle(os.path.join(path,"features_imputation"))

with open("imputation","wb") as f :
    pickle.dump(f_mean,f)

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

X_train = train.loc[:, features]
#y_train = (train.loc[:, 'action'])

y_train = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T

# multimodel

In [ ]:
# modeling step 
params={"num_leaves":300,
       "max_bin":450,
       "feature_fraction":0.52,
       "bagging_fraction":0.52,
       "objective":"binary",
       "learning_rate":0.05,
       "boosting_type":"gbdt",
       "metric":"auc"
       }
models = [] # list of model , we will train 
for i in tqdm(range(y_train.shape[1])):
    xtr,xval,ytr,yval = train_test_split(X_train ,y_train[:,i],test_size=0.2,stratify=y_train[:,i])
   
    d_train = lgbm.Dataset(xtr,label=ytr)
    d_eval = lgbm.Dataset(xval,label=yval,reference=d_train)
    clf = lgbm.train(params,d_train,valid_sets=[d_train,d_eval],num_boost_round=1500,\
                    early_stopping_rounds=50,verbose_eval=50)
    nom_fich = "weights_target_" + resp_cols[i]
    clf.save_model(nom_fich)
    models.append(clf)

# single_model

In [ ]:
params={"num_leaves":300,
       "max_bin":450,
       "feature_fraction":0.52,
       "bagging_fraction":0.52,
       "objective":"binary",
       "learning_rate":0.05,
       "boosting_type":"gbdt",
       "metric":"auc"
       }
sing_model = []
for i in range(3):
  
    clf = lgbm.Booster(model_file=os.path.join(path,"weights_{}".format(i)))
    sing_model.append(clf)

In [ ]:
f = np.median
th = 0.5000
import janestreet
env = janestreet.make_env()
for (test_df, pred_df) in tqdm(env.iter_test()):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        if np.isnan(x_tt.sum()):
            x_tt = np.nan_to_num(x_tt) + np.isnan(x_tt) * f_mean
        # multimodel
        pred1 = np.mean(np.stack([model.predict(x_tt) for model in models]),axis=0).T
        # single_model
        pred2 = np.mean(np.stack([model.predict(x_tt) for model in sing_model]),axis=0)
        
        pred = 0.5 * pred1 + 0.5 * pred2
        pred_df.action = np.where(pred >= th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)